# Tarea 2 - Funciones hash y cifrado por bloques

1. Se recomienda hacer en equipo (de hasta 4 personas) para poder discutir los problemas y repartir el trabajo.
2. Para tu entrega puedes realizar los programas en esta libreta de Jupyter o puedes anexar tus programas en archivos separados. En cualquier caso, agrega comentarios que expliquen el funcionamiento.
3. Cualquier tarea plagiada total o parcialmente se calificará con cero. 

## 1. Vacuna para ransomware (1.5 puntos)

Extrae el archivo `mis_archivos.zip`, que contiene el directorio `Mis archivos`. Desde este directorio ejecuta `juego.py` con Python 3:
```		
Mis archivos$ python3 juego.py
```
Este programa es un ransomware de juguete, encripta todos los archivos del directorio actual y pide una recompensa para poder recuperar los originales. Haz un programa que funcione como vacuna para el ransomware, es decir, que revierta los cambios que fueron hechos al ejecutar `juego.py`. Cuando se ejecute tu programa en un directorio donde anteriormente se ejecutó el ransomware, se recuperarán los archivos originales (tal como eran, sin ninguna diferencia). Explica y justifica el funcionamiento de tu programa vacuna.

## Funciones hash

In [ ]:
from hashlib import sha1

datos = b'datos datos xD'

print('Digesto como bytes')
digesto = sha1(datos).digest()
print(digesto)

print('\nDigesto en hexadecimal')
digesto_hex = sha1(datos).hexdigest()
print(digesto_hex)

El programa vacuna se encuentra en el archivo `vacuna.py` dentro de este directorio. Para recuperar los archivos, hacer lo siguiente:
1. Copiar el archivo `vacuna.py` dentro de la carpeta con los archivos encriptados
2. Ejecutar `python vacuna.py`
3. Seguir las instrucciones del programa

La vacuna sigue los mismos pasos que el archivo `juego.py` pero en sentido inverso. A grandes rasgos hace lo siguiente:
- Recupera los valores guardados en el archivo `.xyz`: c, d y k
- Recupera la llave de cifrado usando k0 = d o k0 = d-1 (esto debido a que en `juego.py`, d se definió como `d |= k`). Primero intenta con k0 = d
- Usa la misma función de cambio de bytes que `juego.py` (como ésta usa XOR, la misma función sirve para descifrar)
- Pregunta al usuario si los archivos fueron recuperados.
 - Si es así, borra los archivos encriptados y termina
 - En otro caso, repite el proceso usando k0 = d - 1

Los detalles linea por linea se encuentran en los comentarios del archivo `vacuna.py`

## 2. Contraseñas en bases de datos (1.5 puntos)

1. El archivo `BD_jaqueada.txt` contiene una lista con los datos de 500 usuarios de un servicio web. La lista incluye el nombre de usuario y el hash de su contraseña. Por la forma de las cadenas en hexadecimal haces la suposición de que cada cadena es de la forma `$salt$H(contraseña||salt)` o `$salt$H(salt||contraseña)`, donde H es una función hash popular con salida de 32 bytes. Usando la lista de contraseñas conocidas `passwords.gz` que se adjunta con este documento, haz un programa para encontrar las contraseñas de los usuarios (por lo menos 50 contraseñas por cada miembro del equipo). Toma en cuenta que la búsqueda puede llevar un buen rato, así que reduce lo más que puedas la cantidad de operaciones realizadas durante la búsqueda. **Advertencia:** el archivo de contraseñas conocidas es muy grande, ten cuidado al usarlo ya que tus programas o incluso tu computadora podrían morir si se acaba la memoria disponible.

2. En otra base de datos se usó el algoritmo `scrypt` para obtener el hash de las contraseñas, pues con este algoritmo se dificulta la búsqueda mediante listas de contraseñas comunes. `scrypt` tiene varios parámetros que se pueden cambiar para incrementar el costo (en procesador y memoria) de calcular un hash. La biblioteca hashlib incluye `scrypt`, pruébalo y después, usando los parámetros $n=2^{16}, r=8, p=1$ y la cadena salt `0xd8201aae236713fefe9a5266dc1f8012`, encuentra la contraseña de juanito. Es posible que tengas que pasar el parámetro `maxmem=2**30`.
```
juanito 5f495364792782144918397bdbb72bc04326a883138a11f3d0b61a3d2576ca00
```

## 3. Minando pumacoins (1.5 puntos)

El pumacoin (PMC) es una criptomoneda (100% real no fake) similar al Bitcoin, pero que en vez de SHA256 utiliza una función hash más moderna llamada BLAKE2. BLAKE2 es una actualización de BLAKE, función hash que participó en la competencia para definir el estándar SHA3 pero que perdió frente a las funciones esponja (Keccak). A pesar de no ser un estándar, la seguridad y velocidad de BLAKE2 han permitido que sea adoptada en bastantes aplicaciones. La biblioteca hashlib de las últimas versiones de Python incluye una implementación de BLAKE2.

Para generar un pumacoin, la especificación del protocolo dice que se usa una *prueba de trabajo* definida de la siguiente manera:

Dado el identificador de un bloque de transacciones `id`, de longitud 16 bytes, los mineros tienen el reto de encontrar una cadena `x` de 16 bytes tal que el digesto generado por `BLAKE2s-256(id||x)` tiene al principio una cadena de bytes especial. Dicha cadena depende de la cantidad de mineros participando en la red: si hay a lo más 100 mineros la cadena especial es `0x242424`, cuando 100 < núm. de mineros < 5000 la cadena  es `0xf09fa491` y cuando hay 5000 o más mineros la cadena especial es `0xe29a92e29898`. El primer minero que encuentra `x` verifica que el bloque es correcto y comunica a los demás mineros que encontró `x`, a este minero se le recompensa con 1 PMC.

1. Implementa una función para minar pumacoins. Recibe como entrada el `id` del bloque y `N` el número de mineros en un momento dado, y como salida da la cadena `x`.
2. Resuelve la prueba de trabajo para las entradas `(0xd1c5593465eb5bfb9fcad9adf90af61f, 50)` y `(0x73bf71c8cd6f03c414cd2477a17570c4, 1000)`. Muestra las cadenas encontradas junto con el digesto que generan.
3. (Opcional) Intenta con la entrada `(0x68188585019b02d746b48b4d06c15dcf, 5000)`, esto llevará más tiempo dependiendo de la capacidad de tu computadora (y de tu suerte).

### 3.1 

In [1]:
import os 
from hashlib import blake2s

def minar_puma(id,N):
    '''
    Mina pumacoins dependiendo el numero de mineros
    Params:
    id: id del bloque 
    N: numero de mineros 
    Return:
    x : cadena
    '''
    id2 = bytes.fromhex(id[2:]) # transformo de hex a bytes
    cadena_especial = ['0x242424', '0xf09fa491', '0xe29a92e29898']
    if N <= 100:
        _minar('242424',id2)
    elif 100 < N < 5000:
        _minar('f09fa491',id2)
    elif N >= 5000:
        _minar('e29a92e29898',id2)

def _minar(cadena,id):
    '''
    Params:
    cadena: cadena especial que deben obtener el minero
    id: id del bloque
    '''
    rand = os.urandom(16) 
    d = id + rand
    x = blake2s(d,digest_size = 16).hexdigest()
    especial = x[0:len(cadena)]
    while especial != cadena:
        d = id + bytes.fromhex(x)
        x = blake2s(d,digest_size = 16).hexdigest()
        especial = x[0:len(cadena)]
    print('id: ',id.hex())
    print('Genera a x: ',d.hex())
    print('x= ',x)       
    return x

### 3.2

Los resultados estan en la imagen pumacoins.png

In [ ]:
minar_puma('0xd1c5593465eb5bfb9fcad9adf90af61f', 50) #40s aprox.
print('*'*64)
minar_puma('0x73bf71c8cd6f03c414cd2477a17570c4', 1000) # 90 min aprox.

Resultados..............................................


<img src= pumacoins.png>


## Modos de operación en cifrado por bloques

Recordemos que los cifradores por bloques solamente funcionan con mensajes de una longitud fija (longitud de bloque). Para poder encriptar un mensaje de longitud arbitraria lo que se hace es
1. Agregar un relleno al final del mensaje para que su longitud sea un múltiplo de la longitud de bloque.
2. Dividir el mensaje en bloques.
3. Aplicar el algoritmo de cifrado.

El paso 3 puede hacerse de distintas maneras, a cada de una de ellas le llamamos **modo de operación**. Existen muchos modos de operación, varios de ellos están definidos en estándares, pero en esta tarea solamente abordaremos tres de ellos.

#### Modo ECB

El modo de operación más directo e *inseguro* se conoce como Electronic Codebook o ECB. Consiste en inicializar el algoritmo con la llave *k* y aplicarlo en cada bloque del mensaje. Más detalladamente, si $E_k$ es nuestro algoritmo cifrador inicializado con la llave $k$ y tenemos el mensaje $m = m_1m_2\cdots m_n$, donde cada $m_i$ es un bloque, obtenemos el mensaje cifrado $c = c_1c_2\cdots c_n$ donde $c_i = E_k(m_i)$.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d6/ECB_encryption.svg/600px-ECB_encryption.svg.png "Modo ECB")

En clase vimos que un algoritmo cifrador determinista no es seguro, ya que al aplicarlo a mensajes claros iguales se obtienen mensajes cifrados iguales. De la misma manera, este modo de operación mantiene los patrones que se pueden encontrar en los bloques del mensaje claro. Para arreglar este problema, los modos de operación posteriores utilizan una cadena extra que permite *aleatorizar* el mensaje.

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB

key = os.urandom(16)
#key = b'Mi llave secreta'
# Se inicializa AES con la llave y el modo ECB
aes_k = Cipher(AES(key), ECB())
enc = aes_k.encryptor()
# Mensaje de tres bloques de longitud
mensaje = b'un bloke secreto' + b'-'*16 + b'un bloke secreto'
mensaje_cifrado = enc.update(mensaje) + enc.finalize()
print('Bloques cifrados')
print(mensaje_cifrado[:16].hex(), mensaje_cifrado[16:32].hex(), mensaje_cifrado[32:].hex())

dec = aes_k.decryptor()
mensaje_descifrado = dec.update(mensaje_cifrado) + dec.finalize()
print('Mensaje descifrado')
print(mensaje_descifrado)

#### Modo CBC
El modo de operación Cipher block chaining o CBC usa un *vector de inicialización* $IV$ para aleatorizar el mensaje claro. Para encriptar $m = m_1m_2\cdots m_n$, definimos $c_0 = IV$ y obtenemos el mensaje cifrado $c = c_0c_1c_2\cdots c_n$, donde $c_i = E_k(m_i\oplus c_{i-1})$; nota que $IV$ se agrega al inicio del mensaje cifrado para que pueda ser usado a la hora de descifrar.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/CBC_encryption.svg/640px-CBC_encryption.svg.png "Modo CBC")

In [ ]:
from cryptography.hazmat.primitives.ciphers.modes import CBC

key = os.urandom(16)
iv = os.urandom(16)
# Se inicializa AES con la llave y el modo CBC, que requiere un IV
aes_k = Cipher(AES(key), CBC(iv))
enc = aes_k.encryptor()
mensaje = b'un bloke secreto' + b'-'*16 + b'un bloke secreto'
mensaje_cifrado = enc.update(mensaje) + enc.finalize()
print('Bloques cifrados')
print(mensaje_cifrado[:16].hex(), mensaje_cifrado[16:32].hex(), mensaje_cifrado[32:].hex())

dec = aes_k.decryptor()
mensaje_descifrado = dec.update(mensaje_cifrado) + dec.finalize()
print('Mensaje descifrado')
print(mensaje_descifrado)

#### Modo CTR

En el modo CTR (de contador) el algoritmo de cifrado se aplica a una cadena $IV||ctr$, donde $IV$ es un vector de inicialización generado aleatoriamente y $ctr$ es una cadena que sirve como contador (el símbolo $||$ se refiere a concatenación). La cadena $IV||ctr$ será la entrada de $E_k$ y el valor del contador se incrementa en uno por cada bloque del mensaje claro. El mensaje cifrado es el XOR de los bloques claros y la salida correspondiente de $E_k$, es decir, el mensaje claro $m = m_1m_2\cdots m_n$ se cifra en el mensaje $c = c_0c_1\cdots c_n$, donde $c_0 = IV$ y para los demás bloques $ c_i = E_k(IV||i) \oplus m_i$.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/CTR_encryption_2.svg/640px-CTR_encryption_2.svg.png "Modo CTR")

### Padding
El relleno o padding que se agrega a los bloques para que tengan el tamaño adecuado puede ser arbitrario (en tanto pueda ser quitado sin ambigüedad). En esta tarea usarás el padding que consiste en agregar la secuencia de bytes BB...B de longitud igual a B, donde B es un entero entre 1 y 16. Por ejemplo, si queremos rellenar la cadena `hola mundo` para tener un bloque de 16 bytes, se agregará la cadena de 6 bytes `0x060606060606`, o la cadena `A` se rellena con la cadena `0x0f0f0f0f0f0f0f0f0f0f0f0f0f0f0f`.

## 4. Implementando modos de operación (1.5 puntos)

Investiga el proceso de descifrado para los modos CBC y CTR (puedes tratar de deducirlo revisando los diagramas mostrados arriba). Luego implementa las funciones de cifrado y descifrado de los modos CBC y CTR usando AES con llaves de 128 bits, para eso vas a utilizar el modo ECB como base para poder aplicar $AES_k$ o $AES_k^{-1}$ según corresponda. Tus funciones aceptarán mensajes de tamaño arbitrario, por lo cual tienen que agregar un relleno según sea necesario. Las funciones de descifrado devolverán el texto claro solamente si el padding del mensaje es correcto, en caso contrario lanzarán una excepción.  El objetivo es que tú programes los modos de operación, por lo cual no puedes usar los modos CBC y CTR que vienen incluidos en bibliotecas.

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB

def aes128_cbc_enc(llave, mensaje):
    aes = Cipher(AES(key), ECB())
    enc = aes.encryptor()
    # FALTA IMPLEMENTAR

def aes128_cbc_dec(llave, cifrado):
    # FALTA IMPLEMENTAR
    
def aes128_ctr_enc(llave, mensaje):
    # FALTA IMPLEMENTAR
    
def aes128_ctr_dec(llave, cifrado):
    # FALTA IMPLEMENTAR

In [ ]:
# Para hacer pruebas
llave = b'Mi llave secreta'
mensaje = b'Mensaje de texto mas grande que un bloque y de longitud que si es multiplo de 16'
mensaje2 = b'UwU'

## 5. Ataque del oráculo de padding (4 puntos)

En este ejercicio programarás el ataque del oráculo de padding, usando AES128 en modo CBC. Para lograrlo realiza las siguientes funciones:
		
1. Para simular el oráculo de padding, implementa una función llamada `oraculo_padding` que recibe un criptotexto (con el IV incluido al inicio) y devuelve un valor booleano: verdadero cuando el mensaje descifrado tiene un relleno válido y falso en caso contrario. `oraculo_padding` llamará a tu función `aes128_cbc_dec` para descifrar el criptotexto, usando la llave `k` que está definida fuera de `oraculo_padding`. Nota que `k` no es parte de la entrada del oráculo, pues la idea es simular una caja negra que solamente responde si el descifrado tuvo un padding correcto.

In [ ]:
# La llave k que usa el oráculo
k = LLAVE

# El oráculo intenta desencriptar y devuelve True si el padding es correcto.
def oraculo_padding(criptotexto):
    # FALTA

2. Haz una función llamada `recupera_padding` que recibe una cadena $ C $ de $ 16(n+1) $ bytes, que equivale a un bloque IV y $ n $ bloques de datos, y suponemos que $ M =  AES_k^{-1}(C) $ contiene un padding de entre 1 y 16 bytes. La función devolverá el padding del mensaje. Esta función no puede usar ningún método de AES ni la llave $ k $, pero puede llamar a `oraculo_padding`.

In [ ]:
# Con esta función el adversario puede obtener el relleno que fue agregado al texto claro
# El adversario escoge el criptotexto y puede llamar a oraculo_padding
def recupera_padding(criptotexto):
    # FALTA

3. Implementa una función llamada `recupera_mensaje_original` que recibe una cadena $ C $ de $ 16(n+1) $ bytes, que equivale a un bloque IV y $ n $ bloques de datos. La función devuelve $ AES_k^{-1}(C) $ usando el modo CBC, es decir, recupera el mensaje claro que corresponde al mensaje cifrado $ C $.  Esta función no puede usar ningún método de AES ni la llave $ k $, pero puede llamar a `oraculo_padding` y `recupera_padding`.


In [ ]:
# El adversario descifra el mensaje, puede consultar al oráculo o a recupera_padding
def recupera_mensaje_original(criptotexto):
    # FALTA